In [2]:
from google.cloud import bigquery
import os
import tweepy
import pandas as pd
import numpy as np
import csv
from tweepy import OAuthHandler
from datetime import datetime
from datetime import timedelta

import sys
sys.path.append(r'C:\Users\amand\project')
import twitterkey2

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=r'C:\Users\amand\project\RA_WORK_CONTENT_BQ.json'



search->sentiment->reply->follower->following

search + industry = collect

search + sentiment + reply = yilin 

check what following mean




In [3]:

auth = OAuthHandler( twitterkey2.consumer_key, twitterkey2.consumer_secret)
auth.set_access_token(twitterkey2.access_token, twitterkey2.access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

### def store function

In [4]:
def implicit():
    from google.cloud import storage
    storage_client = storage.Client()
    buckets = list(storage_client.list_buckets())
    print(buckets)

client = bigquery.Client()
dataset_id = 'ceo_tweet_streaming'  
table_id = 'search_2017_14' 
table_ref = client.dataset(dataset_id).table(table_id)
table = client.get_table(table_ref) 

table_id_hm = 'hashtag_mention'
table_ref_hm = client.dataset(dataset_id).table(table_id_hm)
table_hm = client.get_table(table_ref_hm) 
 
def store_data(	name, id_str, status_id,  CO_PER_ROL, YEAR, EXEC_FULLNAME, created_at, fulltext, truncated, retweet_cnt, favorite_cnt, 
               reply_to_status_id, reply_to_name, reply_to_user_id, 
               is_quote, quote_status_id, quote_created_at, quote_user_id, quote_user_name, quote_user_screen_name, quote_user_description,
               is_retweet, retweet_status_id, retweet_created_at, retweet_user_id, retweet_user_name, retweet_user_screen_name, retweet_user_description):
    rows_to_insert = [(name, id_str, status_id, CO_PER_ROL, YEAR, EXEC_FULLNAME,created_at, fulltext, truncated, retweet_cnt, favorite_cnt, 
                       reply_to_status_id, reply_to_name,reply_to_user_id, 
                       is_quote, quote_status_id, quote_created_at, quote_user_id, quote_user_name, quote_user_screen_name, quote_user_description,
                       is_retweet, retweet_status_id, retweet_created_at, retweet_user_id, retweet_user_name, retweet_user_screen_name, retweet_user_description),]  
    errors = client.insert_rows(table, rows_to_insert)
    print(errors, 'bq')
    assert errors == []
    return

def store_data_hm(status_id, hashtag, mention):
        rows_to_insert_hm = [(status_id, hashtag, mention), ]
        errors_hm = client.insert_rows(table_hm, rows_to_insert_hm)
        print(errors_hm, 'bq_hm')    
        assert errors_hm == []
        return


In [5]:

ceo = pd.read_csv(r'ceo.csv', engine = "python", index_col=False, skiprows = 1, 
    encoding ="ISO-8859-1", header=None, na_values = '-')
ceo.columns  = ['CO_PER_ROL',  'EXEC_FULLNAME', 'YEAR', 'T_NAME']

ceo1 = ceo[0:100]
ceo2 = ceo[100:200]
ceo3 = ceo[200:300]
ceo4 = ceo[300: ]
ceo.head()

,CO_PER_ROL,EXEC_FULLNAME,YEAR,T_NAME
0,45435,Abraham N. Reichental,2015,AviReichental
1,39584,Ahmad R. Chatila,2014,AhmadRChatila
2,31477,Angel R. Martinez,2015,AngelSB55
3,38281,Anthony Aquila,2015,TonyAquila8020
4,34056,Bahram Akradi,2014,bahramakradi


### search

In [11]:

ceo_cnt = 0
fail = 0
cnt = 0

for row in ceo.itertuples():
    name = row.T_NAME
    ceo_cnt += 1
    try:
        tweet = tweepy.Cursor(api.user_timeline, screen_name=name, tweet_mode='extended', wait_on_rate_limit =True, wait_on_rate_limit_notify=True).items()
        if tweet == []:
            continue
            
        else: 
            for line in tweet:
                if cnt == 20:
                    break
                cnt += 1
                is_retweet = False
                full_text = line.full_text
                clean_timestamp = line.created_at
                retweet_status_id = None
                retweet_created_at = None
                retweet_user_id = None
                retweet_user_name = None
                retweet_user_screen_name = None
                retweet_user_description = None
                quote_status_id = None
                quote_created_at = None
                quote_user_id = None
                quote_user_name = None
                quote_user_screen_name = None
                quote_user_description = None
                if hasattr(line, 'retweeted_status'):
                    is_retweet = True
                    full_text = line.retweeted_status.full_text 
                    retweet_status_id = line.retweeted_status.id
                    retweet_created_at = line.retweeted_status.created_at
                    retweet_user_id = line.retweeted_status.user.id
                    retweet_user_name = line.retweeted_status.user.screen_name
                    retweet_user_screen_name = line.retweeted_status.user.name
                    retweet_user_description = line.retweeted_status.user.description
                if hasattr(line, 'quoted_status'):
                    quote_status_id = line.quoted_status_id
                    quote_created_at = line.quoted_status.created_at
                    quote_user_id = line.quoted_status.user.id
                    quote_user_name = line.quoted_status.user.screen_name
                    quote_user_screen_name = line.quoted_status.user.name
                    quote_user_description = line.quoted_status.user.description
                    
                hashtag = [h['text'] for h in line.entities['hashtags']]
                mention = [ str(j['id']) + "," + str(j['screen_name']) + "," + str(j['name']) for j in line.entities['user_mentions']]
                
                
                store_data(name, line.user.id, line.id, row.CO_PER_ROL, row.YEAR, row.EXEC_FULLNAME, clean_timestamp, full_text, line.truncated, line.retweet_count, line.favorite_count,  
                          line.in_reply_to_status_id, line.in_reply_to_screen_name, line.in_reply_to_user_id, 
                          line.is_quote_status, quote_status_id, quote_created_at, quote_user_id, quote_user_name, quote_user_screen_name, quote_user_description,
                          is_retweet, retweet_status_id, retweet_created_at, retweet_user_id, retweet_user_name, retweet_user_screen_name, retweet_user_description)
                store_data_hm(line.id, hashtag, mention)
    except tweepy.TweepError:
        fail += 1
        
print('search ceo :' + str(ceo_cnt))
print('search fail :' + str(fail))
print('return search result :' + str(cnt))

[] bq_hm
[] bq_hm
[] bq_hm
[] bq_hm
[] bq_hm
[] bq_hm
[] bq_hm
[] bq_hm
[] bq_hm
[] bq_hm
[] bq_hm
[] bq_hm
[] bq_hm
[] bq_hm
[] bq_hm
[] bq_hm
[] bq_hm
[] bq_hm
[] bq_hm
[] bq_hm


KeyboardInterrupt: 

### tb_id

In [12]:
file = open(r'tweet_id.csv', 'a', encoding="utf-8", newline='')
csvwriter = csv.writer(file)
csvwriter.writerow(['name', 'id_str', 'CO_PER_ROL', 'YEAR', 'EXEC_FULLNAME', 'followers_count', 'friends_count', 'lang', 'account_created', 'location', 'verified'])

#key = id
tweetCount = 1
ceo_cnt = 0
tweet_cnt = 0
no_twitter_cnt = 0
fail = 0

for row in ceo.itertuples():
    name = row.T_NAME
    ceo_cnt += 1

    try:
        tweet = api.user_timeline(screen_name=name, count=tweetCount, wait_on_rate_limit_notify=True, wait_on_rate_limit=True)
        if tweet == []:
            no_twitter_cnt += 1
            continue
        else: 
            tweet_cnt += 1
            for line in tweet:
               csvwriter.writerow([name,"'" + str(line.user.id_str), row.CO_PER_ROL, row.YEAR, row.EXEC_FULLNAME, line.user.followers_count, line.user.friends_count, line.lang, line.user.created_at, line.user.location, line.user.verified])
    except tweepy.TweepError:
        fail += 1

    # if tweet_cnt == 15:
        # break

print('search ceo :' + str(ceo_cnt))
print('search has no result :' + str(no_twitter_cnt))
print('return search result :' + str(tweet_cnt))


file.close()

search ceo :361
search has no result :40
return search result :288
